# Prediction des défauts de paiement de cartes de crédit

Nous allons utiliser le jeu de données "Default of Credit Card Clients" disponible sur le site [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients). Source: Yeh, I. C., & Lien, C. H. (2009). The comparisons of data mining techniques for the predictive accuracy of probability of default of credit card clients. Expert Systems with Applications, 36(2), 2473-2480.

Ce jeu de données contient des informations sur les défauts de paiement, les facteurs démographiques, les données de crédit, l'historique des paiements et les factures de cartes de crédit de 30 000 clients à Taiwan d'avril 2005 à septembre 2005.

Le jeu de données contient 25 variables :

- **ID** : ID client
- **LIMIT_BAL** : Montant du crédit accordé en dollar NT
- **SEX** : Le genre du client (1 = homme, 2 = femme)
- **EDUCATION** : Niveau d'éducation (1 = cycle master/doctorat, 2 = université, 3 = lycée, 4 = autre)
- **MARRIAGE** : Etat matrimonial (1 = marié, 2 = célibataire, 3 = autre)
- **AGE** : Age
- **PAY_0** : Etat de remboursement en septembre 2005 (-1 = paiement effectué, 1 = paiement en retard de 2 mois, ..., 9 = paiement en retard de 9 mois et plus)
- **PAY_2** : Etat de remboursement en août 2005
- **PAY_3** : Etat de remboursement en juillet 2005
- **PAY_4** : Etat de remboursement en juin 2005
- **PAY_5** : Etat de remboursement en mai 2005
- **PAY_6** : Etat de remboursement en avril 2005
- **BILL_AMT1** : Montant de la facture en septembre 2005 (en dollar NT)
- **BILL_AMT2** : Montant de la facture en août 2005
- **BILL_AMT3** : Montant de la facture en juillet 2005
- **BILL_AMT4** : Montant de la facture en juin 2005
- **BILL_AMT5** : Montant de la facture en mai 2005
- **BILL_AMT6** : Montant de la facture en avril 2005
- **PAY_AMT1** : Montant du paiement précédent en septembre 2005 (en dollar NT)
- **PAY_AMT2** : Montant du paiement précédent en août 2005
- **PAY_AMT3** : Montant du paiement précédent en juillet 2005
- **PAY_AMT4** : Montant du paiement précédent en juin 2005
- **PAY_AMT5** : Montant du paiement précédent en mai 2005
- **PAY_AMT6** : Montant du paiement précédent en avril 2005
- **default.payment.next.month** : Défaut de paiement en mai 2005 (1 = oui, 0 = non)
